In [20]:
# import os
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin

# # Configuration
# BASE_URL = "https://www.cms.gov/priorities/innovation/evaluation-research-reports"  # Replace with your target URL
# SAVE_DIR = "data/raw/healthcare"
# os.makedirs(SAVE_DIR, exist_ok=True)  # Create directory if missing

# # Custom headers to mimic browser behavior
# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
# }

# def sanitize_filename(filename):
#     """Remove invalid characters from filenames"""
#     return "".join(c for c in filename if c.isalnum() or c in (' ', '-', '_')).rstrip()

# def download_pdf(url, save_path):
#     """Download PDF with error handling"""
#     try:
#         response = requests.get(url, headers=HEADERS, timeout=10)
#         response.raise_for_status()  # Raise HTTP errors
#         with open(save_path, 'wb') as f:
#             f.write(response.content)
#         print(f"✅ Saved: {save_path}")
#         return True
#     except Exception as e:
#         print(f"❌ Failed to download {url}: {str(e)}")
#         return False

# # Main scraping function
# def scrape_and_download():
#     print(f"🔍 Scraping {BASE_URL}...")
#     try:
#         response = requests.get(BASE_URL, headers=HEADERS)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         pdf_count = 0
#         for link in soup.find_all('a', href=True):
#             href = link['href']
            
#             # Skip non-PDF links
#             if not href.lower().endswith('.pdf'):
#                 continue
                
#             # Construct absolute URL
#             pdf_url = urljoin(BASE_URL, href)
            
#             # Generate filename from link text or URL
#             filename = sanitize_filename(link.text.strip() or pdf_url.split('/')[-1]) + ".pdf"
#             save_path = os.path.join(SAVE_DIR, filename)
            
#             # Download PDF
#             if download_pdf(pdf_url, save_path):
#                 pdf_count += 1
        
#         print(f"🎉 Downloaded {pdf_count} PDFs to {SAVE_DIR}")
        
#     except Exception as e:
#         print(f"⚠️ Scraping failed: {str(e)}")

# if __name__ == "__main__":
#     scrape_and_download()

In [21]:
# import os
# import re
# import time
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin
# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


# # Configuration - works in notebooks/IDEs/standalone scripts
# try:
#     BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(__file__)), "data", "raw")
# except NameError:
#     BASE_DIR = os.path.join(os.getcwd(), "data", "raw")  # Fallback for notebooks
# CATEGORIES = {
#     "healthcare": [
#         "https://www.cms.gov/about-cms/agency-information/history",
#         "https://www.medicaid.gov/about-us/reports-and-evaluations/index.html"
#     ],
#     "defense": [
#         "https://www.defense.gov/News/Publications/",
#         "https://www.esd.whs.mil/DD/",
#         "https://media.defense.gov/Publications/"
#     ],
#     "education": [
#         "https://www2.ed.gov/about/reports/annual/index.html",
#         "https://nces.ed.gov/pubsearch/",
#         "https://www2.ed.gov/policy/gen/leg/foia/library.html"
#     ],
#     "finance": [
#         "https://www.federalreserve.gov/publications.htm",
#         "https://home.treasury.gov/policy-issues/financial-markets-financial-institutions-and-fiscal-service/fsoc/reports",
#         "https://www.sec.gov/reports.shtml"
#     ]
# }
# # HEADERS = {
# #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
# # }
# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
#     'Accept': 'application/pdf, text/html'
# }
# DELAY = 2  # Conservative delay for .gov sites

# def setup_project_folders():
#     """Create ./data/raw/[category] structure in current project"""
#     for category in CATEGORIES:
#         category_path = os.path.join(BASE_DIR, category)
#         os.makedirs(category_path, exist_ok=True)
#         print(f"📁 Created: {os.path.relpath(category_path)}")

# def is_gov_pdf(url):
#     """Enhanced PDF detection for government sites"""
#     if not url:
#         return False
#     url_lower = url.lower()
#     return (
#         url_lower.endswith('.pdf') or
#         '/pdf' in url_lower or
#         any(x in url_lower for x in [
#             'download=pdf',
#             'type=pdf',
#             '/document_library/',
#             '/publications/',
#             'file=pdf'
#         ])
#     )

# def download_gov_pdf(url, save_path):
#     """Government-specific downloader with robust handling"""
#     try:
#         response = requests.get(
#             url,
#             headers=HEADERS,
#             stream=True,
#             verify=False,
#             timeout=30
#         )
#         response.raise_for_status()

#         # Handle Content-Disposition filenames
#         if 'Content-Disposition' in response.headers:
#             filename = re.findall(
#                 'filename="?([^"]+)"?',
#                 response.headers['Content-Disposition']
#             )[0]
#             save_path = os.path.join(os.path.dirname(save_path), filename)

#         with open(save_path, 'wb') as f:
#             for chunk in response.iter_content(8192):
#                 f.write(chunk)
        
#         print(f"✅ Saved: {os.path.relpath(save_path)}")
#         return True

#     except Exception as e:
#         print(f"❌ Failed to download {os.path.basename(url)}: {str(e)}")
#         return False

# def scrape_gov_site(base_url):
#     """Specialized scraper for government document portals"""
#     try:
#         time.sleep(DELAY)
#         response = requests.get(base_url, headers=HEADERS, verify=False, timeout=30)
#         response.raise_for_status()
#         # 
#         soup = BeautifulSoup(response.text, 'html.parser')
#         pdf_links = set()  # Avoid duplicates

#         # Government sites often use these patterns
#         for link in soup.find_all('a', href=True):
#             href = urljoin(base_url, link['href'])
#             if is_gov_pdf(href):
#                 pdf_links.add(href)

#         return sorted(pdf_links)[:10]  # Return first 10 unique PDFs

#     except Exception as e:
#         print(f"⚠️ Scraping error at {base_url}: {str(e)}")
#         return []

# def main():
#     print("\n=== Government Document Collector ===")
#     print(f"📂 Root directory: {os.path.abspath(BASE_DIR)}\n")
#     setup_project_folders()

#     for category, urls in CATEGORIES.items():
#         print(f"\n🔍 Processing {category.upper()} documents:")
        
#         for url in urls:
#             print(f"   - Scanning: {url}")
#             pdfs = scrape_gov_site(url)
            
#             if not pdfs:
#                 print("   ⚠️ No PDFs found using standard methods")
#                 continue
                
#             category_dir = os.path.join(BASE_DIR, category)
#             for pdf_url in pdfs:
#                 # Generate clean filename
#                 filename = (
#                     os.path.basename(pdf_url.split('?')[0])
#                     or f"{category}_{int(time.time())}.pdf"
#                 )
#                 # Replace spaces and special chars
#                 filename = re.sub(r'[^\w\-.]', '_', filename)
                
#                 save_path = os.path.join(category_dir, filename)
#                 download_gov_pdf(pdf_url, save_path)
#                 time.sleep(DELAY)

# if __name__ == "__main__":
#     main()
#     print("\n🏁 Collection complete! Files saved to project's data/raw folder")

In [33]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, unquote
import time
from fake_useragent import UserAgent

# Configuration
SEARCH_URL = "PASTE_GOOGLE_SEARCH_URL_HERE"  # e.g. "https://www.google.com/search?q=site:defense.gov+filetype:pdf+report"
DOWNLOAD_FOLDER = "data/raw/test"  # Changed to your requested path
MAX_PDFS = 5
DELAY = 2  # seconds between downloads

def setup():
    """Create download folder with subdirectories"""
    os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)
    print(f"📁 Created download folder at: {os.path.abspath(DOWNLOAD_FOLDER)}")

def get_random_headers():
    """Generate rotating headers to avoid blocking"""
    ua = UserAgent()
    return {
        'User-Agent': ua.random,
        'Accept': 'text/html,application/pdf',
        'Referer': 'https://www.google.com/'
    }

def extract_pdf_links(url):
    """Improved PDF link extraction"""
    print(f"\n🔍 Scraping Google results from: {url[:80]}...")
    try:
        response = requests.get(
            url,
            headers=get_random_headers(),
            timeout=15
        )
        soup = BeautifulSoup(response.text, 'html.parser')
        
        pdf_links = []
        for a in soup.select('a[href^="/url?q="]'):  # Google result links
            try:
                raw_link = a['href'].split('&')[0]
                link = unquote(raw_link.replace('/url?q=', ''))
                if link.lower().endswith('.pdf'):
                    pdf_links.append(link)
            except:
                continue
                
        return list(set(pdf_links))[:MAX_PDFS]  # Remove duplicates and limit
    
    except Exception as e:
        print(f"❌ Scraping failed: {str(e)}")
        return []

def download_pdf(url):
    """More reliable PDF downloader"""
    try:
        # Generate filename
        filename = os.path.basename(urlparse(url).path)
        if not filename.lower().endswith('.pdf'):
            filename = f"document_{int(time.time())}.pdf"
        
        save_path = os.path.join(DOWNLOAD_FOLDER, filename)
        
        print(f"⬇️ Downloading: {filename[:50]}...")
        response = requests.get(
            url,
            headers=get_random_headers(),
            stream=True,
            timeout=30,
            verify=False  # Bypass SSL verification for .gov/.mil sites
        )
        response.raise_for_status()
        
        # Stream download with progress
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(8192):
                f.write(chunk)
        
        # Verify download size
        file_size = os.path.getsize(save_path) / 1024  # KB
        if file_size < 5:  # Skip very small files
            os.remove(save_path)
            print(f"❌ Removed small file (only {file_size:.1f} KB)")
            return False
            
        print(f"✅ Saved {filename} ({file_size:.1f} KB)")
        return True
        
    except Exception as e:
        print(f"❌ Download failed: {str(e)}")
        return False

def main():
    setup()
    pdf_links = extract_pdf_links(SEARCH_URL)
    
    if not pdf_links:
        print("\n⚠️ No PDF links found. Possible issues:")
        print("- Google is blocking the scraper")
        print("- No PDF results on the search page")
        print("- Try a different search query")
        return
    
    print(f"\nFound {len(pdf_links)} PDFs:")
    for i, url in enumerate(pdf_links, 1):
        print(f"{i}. {url[:80]}...")
    
    print("\nStarting downloads...")
    for url in pdf_links:
        download_pdf(url)
        time.sleep(DELAY)  # Be polite to servers

if __name__ == "__main__":
    main()
    print(f"\n🏁 Check your downloads in: {os.path.abspath(DOWNLOAD_FOLDER)}")

📁 Created download folder at: /Users/gwin/Documents/Post Undergrad Work/Tax Search/data/raw/test

🔍 Scraping Google results from: PASTE_GOOGLE_SEARCH_URL_HERE...
❌ Scraping failed: Invalid URL 'PASTE_GOOGLE_SEARCH_URL_HERE': No scheme supplied. Perhaps you meant https://PASTE_GOOGLE_SEARCH_URL_HERE?

⚠️ No PDF links found. Possible issues:
- Google is blocking the scraper
- No PDF results on the search page
- Try a different search query

🏁 Check your downloads in: /Users/gwin/Documents/Post Undergrad Work/Tax Search/data/raw/test


In [35]:
from pdfminer.high_level import extract_text
import os

def pdf_to_text(folder_path):
    texts = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            filepath = os.path.join(folder_path, filename)
            texts[filename] = extract_text(filepath)
    return texts

In [36]:
categories = ['healthcare', 'defense', 'finance', 'education']
raw_data = {category: pdf_to_text(f'data/raw/{category}') for category in categories}

PDFSyntaxError: No /Root object! - Is this really a PDF?